### 新建电影库和标签库

In [ ]:
mdata=pd.DataFrame(columns=list(['名称','年份','评分','评论数','影评数','标签','链接地址']))


In [72]:
mtag=pd.DataFrame(columns=list(['标签','链接地址']))

### 定义添加标签方法

In [48]:
'''
添加标签
'''
def add_tag(tag,url):
    if tag!='':
        mtag.loc[len(mtag)]={'标签':tag,'链接地址':url}
    

### 定义相关爬取方法

In [49]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re

headers = {
    'User-Agent':'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36 QIHU 360SE'
}

'''
获取详情相关数据
'''
def get_movie_details(url):
    try:
        mhtml=requests.get(url,headers=headers)
        soup=BeautifulSoup(mhtml.text,'html.parser')
        '''
        名称
        '''
        name= soup.find('span',property="v:itemreviewed").text
        year= soup.find('span',class_="year").text.replace('(','').replace(')','')
        score =soup.find('strong',class_="ll rating_num").text
        commentCount=soup.find('a',href=url+"comments?status=P").text.replace(' ','').replace('全部','').replace('条','')
         
        tag=div_tag=soup.find('div',class_='tags-body').text

        reviewCount=soup.find_all('a',href="reviews")[0].text.replace(' ','').replace('全部','').replace('条','')
        # return {'名称':name,'年份':year,'评分':score,'评论数':commentCount,'影评数':reviewCount,'链接地址':url}
        return {'名称': name, '年份': year, '评分': score, '评论数': commentCount, '影评数': reviewCount,'标签':tag,'链接地址':url}
    except(Exception) as err:
        print(err)
    
    return {'名称': '', '年份': '', '评分': '', '评论数': '', '影评数': '','标签':'','链接地址':url}

In [39]:
a=get_movie_details('https://movie.douban.com/subject/30327842/')
a

{'名称': '82年生的金智英 82년생 김지영',
 '年份': '2019',
 '评分': '8.6',
 '评论数': '89384',
 '影评数': '2242',
 '标签': '\n女性\n韩国\n家庭\n社会\n平权\n人生\n女权\n剧情\n',
 '链接地址': 'https://movie.douban.com/subject/30327842/'}

In [41]:
# type(a['标签'].split('\n'))

list

In [43]:
# for tmp_tag in a['标签'].split('\n'):
#     add_tag(tmp_tag,'')

In [53]:
import json

flag=True
pageIndxe=0
while(flag):
    mlist=requests.get('https://movie.douban.com/j/search_subjects?type=movie&tag=%E7%83%AD%E9%97%A8&sort=rank&page_limit=20&page_start='+str(pageIndxe),headers=headers)
    mdict = json.loads(mlist.text)
    mplist=mdict["subjects"]
    if len(mplist)==0:
        flag=False
    pageIndxe=pageIndxe+20

    for mp in mplist:
        m=get_movie_details(mp["url"])
        # 添加电影数据
        mdata.loc[len(mdata)]=m
        
        # 添加标签数据
        for tmp_tag in m['标签'].split('\n'):
            add_tag(tmp_tag,mp["url"])
    

        print("采集中："+mp["url"])
print("完成") 

ie.douban.com/subject/30271841/
采集中：https://movie.douban.com/subject/30318116/
采集中：https://movie.douban.com/subject/30292777/
采集中：https://movie.douban.com/subject/30331959/
采集中：https://movie.douban.com/subject/26709258/
采集中：https://movie.douban.com/subject/30410114/
采集中：https://movie.douban.com/subject/26858510/
采集中：https://movie.douban.com/subject/30211998/
采集中：https://movie.douban.com/subject/26871906/
采集中：https://movie.douban.com/subject/27132510/
采集中：https://movie.douban.com/subject/26348103/
采集中：https://movie.douban.com/subject/25842038/
采集中：https://movie.douban.com/subject/32659890/
采集中：https://movie.douban.com/subject/30241298/
采集中：https://movie.douban.com/subject/27002635/
采集中：https://movie.douban.com/subject/33420285/
采集中：https://movie.douban.com/subject/34805219/
采集中：https://movie.douban.com/subject/30258072/
采集中：https://movie.douban.com/subject/34875369/
采集中：https://movie.douban.com/subject/26786669/
采集中：https://movie.douban.com/subject/34951277/
采集中：https://movie.douban.com

### 抓取完成

In [3]:
mdata.head(10)

NameError: name 'mdata' is not defined

In [87]:
for i in range(0,len(mdata)):
    for tmp_tag in str(mdata.iloc[i]['标签']).split('\n'):
        add_tag(tmp_tag,mdata.iloc[i]['链接地址'])


In [89]:
a=mtag.groupby('标签').size()

In [103]:
a.sort_values(ascending=False).head(30)

标签
2019       242
剧情         191
美国         143
2020       130
犯罪          97
喜剧          85
爱情          72
悬疑          71
电影          64
惊悚          62
动作          60
日本          60
科幻          42
人性          41
动画          38
恐怖          37
韩国          36
温情          36
青春          32
中国大陆        31
英国          31
美国电影        31
女性          30
Netflix     30
日本电影        29
家庭          27
文艺          26
真实事件改编      25
黑色幽默        25
韩国电影        25
dtype: int64

835

In [127]:
mdata.dtypes

名称      object
年份      object
评分      object
评论数      int64
影评数      int64
标签      object
链接地址    object
dtype: object

In [126]:
#mdata['评论数']=mdata['评论数'].astype('int64')

In [129]:
mdata.sort_values('评论数',ascending=False).head(20)

,名称,年份,评分,评论数,影评数,标签,链接地址
4,少年的你,2019,8.3,370640,14772,\n校园霸凌\n青春\n校园暴力\n成长\n校园\n人性\n中国大陆\n2019\n,https://movie.douban.com/subject/30166972/
389,诛仙1 诛仙 Ⅰ,2019,4.5,272320,7915,\n仙侠\n古装\n小说改编\n玄幻\n武侠\n良心\n2019\n中国大陆\n,https://movie.douban.com/subject/25779217/
17,我和我的祖国,2019,7.7,231826,3540,\n建国70周年\n爱国\n正能量\n献礼\n主旋律\n感动\n中国大陆\n2019\n,https://movie.douban.com/subject/32659890/
0,小丑 Joker,2019,8.7,197940,4895,\n人性\n犯罪\nDC\n心理\n社会\n美国\n2019\n暴力\n,https://movie.douban.com/subject/27119724/
69,中国机长,2019,6.7,191433,2294,\n真实事件改编\n航空\n中国\n飞行员\n空难\n灾难\n2019\n迫降\n,https://movie.douban.com/subject/30295905/
177,囧妈,2020,5.9,173307,2937,\n喜剧\n亲情\n2020\n家庭\n温情\n贺岁\n搞笑\n中国大陆\n,https://movie.douban.com/subject/30306570/
6,利刃出鞘 Knives Out,2019,8.2,150434,1707,\n悬疑\n推理\n犯罪\n美国\n侦探\n黑色幽默\n剧情\n反转\n,https://movie.douban.com/subject/30318116/
9,罗小黑战记,2019,8.2,136263,2509,\n治愈\n温情\n动画\n动漫\n萌\n中国大陆\n二次元\n成长\n,https://movie.douban.com/subject/26709258/
31,半个喜剧,2019,7.4,120327,2138,\n开心麻花\n黑色幽默\n喜剧\n爱情\n人性\n成长\n社会\n中国大陆\n,https://movie.douban.com/subject/30269016/
41,冰雪奇缘2 Frozen II,2019,7.2,100825,1614,\n迪士尼\n动画\n童话\n音乐\n美国\n奇幻\n魔幻\n2019\n,https://movie.douban.com/subject/25887288/
